In [6]:
import numpy as np
import pandas as pd
from os import listdir

In [3]:
def findReads(sampleName,sample):
    allReads = []
    for mappingFile in listdir("../4.bam2bed/"):
        if sampleName in mappingFile:
            mappingFile = "../4.bam2bed/" + mappingFile
            mappingFile = pd.read_csv(mappingFile,sep="\t", index_col=0, header=None)
            reads = pd.merge(left=mappingFile,right=sample,left_index=True,right_index=True,how="inner")
            if reads.shape[0]>0:
                allReads.append(reads)
    return pd.concat(allReads)

In [27]:
def getProphages(allReads):
    ppp = allReads[allReads["phageZscore"]>0] #ppp = posible prophages
    prophages = []
    for x in allReads.columns.values[5:-1]:
        ppp1 = ppp[ppp[x]>0]
        regionStart = int(x)*100000
        regionEnd = regionStart + 100000
        ppp2 = ppp1[ppp1[1] > regionStart]
        pp = ppp2[ppp2[1] < regionEnd] #pp = proPhage
        prophages.append(pp)
    if len(prophages)>0:
        return pd.concat(prophages)
    else:
        return 0

In [30]:
sample = pd.read_csv("../testIdentifyGenomesAndRegions/C1.prophagesRegions.csv",sep="\t",index_col=0)
sampleName = "C1"

In [31]:
sample.head()

""


In [28]:
allReads = findReads(sampleName, sample)
allReads["phageZscore"] = allReads[allReads.columns.values[5:]].sum(axis=1)
prophages = getProphages(allReads)
if prophages != 0:
    bacteria = allReads.merge(prophages, indicator=True, how="outer")
    bacteria = bacteria[bacteria["_merge"] == "left_only"]

    out1 = sampleName + ".bacteria.csv"
    bacteria.to_csv(out1,sep='\t')

    out2 = sampleName + ".prophages.csv"
    prophages.to_csv(out2,sep='\t')
else:
    bacteria = allReads
    out1 = sampleName + ".bacteria.csv"
    bacteria.to_csv(out1,sep='\t')